In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
cd '/home/dfried/projects/onecommon/aaai2020/experiments'

/home/dfried/projects/onecommon/aaai2020/experiments


In [4]:
import models
from models import get_model_type
import data
import utils
from domain import get_domain
from corpora.reference_sentence import ReferenceSentenceCorpus

In [5]:
# ls expts/rel3_tsel_ref_dial_model_separate/directions

In [6]:
# ls expts/rel3_tsel_ref_dial_model_separate/directions/base_512_learned-pooling_untied_bidirectional_hidden-mention-encoder/

In [7]:
domain = get_domain('one_common')
# unk_threshold = 10
unk_threshold = 0
model_ty = get_model_type('rnn_reference_model')

ctx_encoder_type = 'rel_attn_encoder_3'

# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

base = 'expts/rel3_tsel_ref_dial_model_separate/directions'
dot_recurrence = 'expts/rel3_tsel_ref_dial_model_separate/dot_recurrence'
dot_recurrence_no_beliefs = 'expts/rel3_tsel_ref_dial_model_separate/dot_recurrence_no-beliefs'

model_fnames = {
    'basic': f'{base}/base_512_learned-pooling_untied_bidirectional_hidden-mention-encoder/1_ep-30.th',
    'encoder-filtered-shared': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-shared/1_ep-30.th',
    'encoder-filtered-separate': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate/1_ep-30.th',
    'encoder-filtered-separate-diffs': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate-diffs/1_ep-30.th',
#     'dot-recurrence': f'{dot_recurrence}/base_dr-self+partner-32-mention-selection/1_ep-20.th',
    'dot-recurrence-no-beliefs': f'{dot_recurrence_no_beliefs}/base_mention-selection-ref-partner/1_ep-20.th',
    'dot-recurrence-oracle-no-beliefs': f'{dot_recurrence_no_beliefs}/base_oracle_mention-selection-ref-partner/1_ep-20.th',
#     'dot-recurrence-nm-multi': f'{dot_recurrence}/base_dr-self+partner-64-mention-selection_nmpt-multi-reference_loss-expanded/1_ep-20.th',
    'dot-recurrence-nm-multi-no-beliefs': f'{dot_recurrence_no_beliefs}/base_mention-selection-ref-partner_nmpt-multi_loss-expanded/1_ep-20.th',
}

In [8]:
seed = 1
corpus = ReferenceSentenceCorpus(domain, 'data/onecommon',
                            train='train_reference_{}.txt'.format(seed),
                            valid='valid_reference_{}.txt'.format(seed),
                            test='test_reference_{}.txt'.format(seed),
                            freq_cutoff=unk_threshold, verbose=True)

freq cutoff: 0
vocab size: 1857
dataset data/onecommon/train_reference_1.txt, total 545056, unks 0, ratio 0.00%
dataset data/onecommon/valid_reference_1.txt, total 69348, unks 202, ratio 0.29%
dataset data/onecommon/test_reference_1.txt, total 67284, unks 210, ratio 0.31%


In [9]:
def load_json(file):
    import json
    with open(file, 'r') as f:
        return json.load(f)

In [10]:
scenarios1 = load_json('data/onecommon/aaai_train_scenarios.json')
scenarios2 = load_json('data/onecommon/aaai_train_scenarios_2.json')

In [11]:
len(scenarios1)

9000

In [12]:
len(scenarios2)

4500

In [13]:
scenarios1[0].keys()

dict_keys(['kbs', 'shared', 'uuid'])

In [14]:
train_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1
}

In [15]:
all_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1 + scenarios2
}

In [16]:
hit_count = 0
miss_count = 0
for scenario in scenarios2:
    uuid = scenario['uuid']
    if uuid in train_scenarios:
        assert train_scenarios[uuid] == scenario
        hit_count += 1
    else:
        miss_count += 1

In [17]:
print("hits: {}".format(hit_count))
print("misses: {}".format(miss_count))

hits: 0
misses: 4500


In [18]:
models = {
    name: utils.load_model(model_fname, map_location='cpu', prefix_dir=None)
    for name, model_fname in model_fnames.items()
}

/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.rnn_reference_model.HierarchicalRnnReferenceModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.ctx_encoder.RelationalAttentionContextEncoder3' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.atte

In [19]:
markable_detector = utils.load_model('serialized_models/markable_detector_1.th', map_location='cpu', prefix_dir=None)
markable_detector.eval()
# TODO: this will no longer be necessary with markable_detector_with_dict
markable_corpus = markable_detector.corpus_ty('one_common', 'data/onecommon',
                                     train='train_markable_1.txt', valid='valid_markable_1.txt', test='test_markable_1.txt')
markable_detector.word_dict = markable_corpus.word_dict

/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.markable_detector.BiLSTM_CRF' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [20]:
batchsize=1
validset, validset_stats = corpus.valid_dataset(batchsize)

100%|██████████████████████████████████████| 1038/1038 [00:01<00:00, 534.74it/s]


In [21]:
validset_stats

{'n': 74030,
 'self_n': 37015,
 'nonpadn': 69348,
 'self_nonpadn': 34674,
 'max_num_mentions': 9}

In [22]:
import torch

In [23]:
from agent import RnnAgent

In [24]:
from selfplay import make_parser

In [25]:
parser = make_parser()
args = parser.parse_args('--temperature 0.25'.split())
# args = parser.parse_args('--temperature 0.01'.split())

In [26]:
def is_selection(out):
    return '<selection>' in out

In [27]:
# display

In [28]:
from dialog import DialogLogger

In [29]:
from IPython.display import SVG, display, HTML

In [30]:
def display_svgs(svgs):
    no_wrap_div = '<div style="white-space: nowrap">' + ''.join(svgs) + '</div>'
    display(HTML(no_wrap_div))

In [31]:
SVG_SCALE=0.4

In [32]:
def display_attn(scenario, attn, agent_id, name=None):
    attn = attn.flatten().detach().numpy()
    if name is not None:
        print("{}: {}".format(name, attn))
    display_svgs([DialogLogger._scenario_to_svg(scenario, scale=SVG_SCALE)[agent_id], DialogLogger._attention_to_svg(scenario, agent_id, attn, scale=SVG_SCALE)])

In [33]:
def display_attns(scenario, attns, agent_id, name=None):
    svgs = [DialogLogger._scenario_to_svg(scenario, scale=SVG_SCALE)[agent_id]]
#     if name is not None:
#         print("{}: {}".format(name, attn))
    for attn in attns:
        attn = attn.flatten().detach().numpy()
        svgs.append(DialogLogger._attention_to_svg(scenario, agent_id, attn, scale=SVG_SCALE))
    if name is not None:
        print(name)
    display_svgs(svgs)

In [34]:
# training emulation

In [35]:
trainset, trainset_stats = corpus.train_dataset(bsz=1)

100%|██████████████████████████████████████| 8304/8304 [00:14<00:00, 582.65it/s]


In [36]:
trainset[0].scenario_ids[0] in train_scenarios

False

In [37]:
# ctx: this player's dots
# inpt / tgt: dialogue word indices (tgt are shifted by 1)
# ref_inpt

In [38]:
for ix in range(len(trainset)):
#     ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
#     [scenario_id], real_ids, agents, chat_ids, sel_idxs, \
#     lens, rev_idxs, hid_idxs, all_num_markables = trainset[ix]
    
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, partner_real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, num_markables, \
    is_self, partner_ref_inpts, partner_ref_tgts_our_view, all_partner_num_markables,\
    ref_disagreements, partner_ref_disagreements = trainset[ix]

    if 'C_d0de3d0144df405e998ae718539ff9a4' in chat_ids:
#     if scenario_id == 'S_UoujG1GG6KscmnbN':
        print("found it")
        scenario = train_scenarios[scenario_id]
        break

found it


In [39]:
models.keys()

dict_keys(['basic', 'encoder-filtered-shared', 'encoder-filtered-separate', 'encoder-filtered-separate-diffs', 'dot-recurrence-no-beliefs', 'dot-recurrence-oracle-no-beliefs', 'dot-recurrence-nm-multi-no-beliefs'])

In [40]:
# model = models['encoder-filtered-separate-diffs']
# model = models['dot-recurrence']
# model = models['dot-recurrence-no-beliefs']
model = models['dot-recurrence-nm-multi-no-beliefs']

In [41]:
from models.reference_predictor import ReferencePredictor

In [42]:
from engines.beliefs import BeliefConstructor

In [43]:
from engines.rnn_reference_engine import make_dots_mentioned_multi, make_dots_mentioned_per_ref_multi

In [44]:
reference_predictor = ReferencePredictor(model.args)

In [45]:
pdb on

Automatic pdb calling has been turned ON


In [47]:
display_mentions = model.args.next_mention_prediction_type == 'multi_reference'

In [48]:
from models.markable_detector import detect_markables

In [66]:
def force_contexts_and_mentions(index, datasplit=validset, candidates=1,
                                sample_temperature=0.25, argmax_temperature=0.005,
                                force_num_markables=False,
                                inference='beam'):
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, partner_real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, num_markables, \
    is_self, partner_ref_inpts, partner_ref_tgts_our_view, partner_num_markables,\
    ref_disagreements, partner_ref_disagreements = datasplit[index]
    
    bsz = ctx.size(0)
    num_dots = 7
    
    # assume we've been training for a while, so that the beliefs are out of the warmup period (e.g. for models that switch from true->predicted)
    epoch = 1000
    
    dots_mentioned = make_dots_mentioned_multi(ref_tgts, model.args, bsz, num_dots)
    dots_mentioned_per_ref = make_dots_mentioned_per_ref_multi(ref_tgts, model.args, bsz, num_dots)

    partner_dots_mentioned_our_view = make_dots_mentioned_multi(
        partner_ref_tgts_our_view, model.args, bsz, num_dots
    )
    partner_dots_mentioned_our_view_per_ref = make_dots_mentioned_per_ref_multi(
        partner_ref_tgts_our_view, model.args, bsz, num_dots
    )
    
    belief_constructor = BeliefConstructor(
        model.args, epoch, bsz, num_dots, inpts, ref_tgts, partner_ref_tgts_our_view,
        real_ids, partner_real_ids, sel_tgt, is_self,
        partner_dots_mentioned_our_view, partner_dots_mentioned_our_view_per_ref,
        dots_mentioned, dots_mentioned_per_ref,
        ref_inpts, partner_ref_inpts,
        num_markables, partner_num_markables,
    )
    
    scenario = all_scenarios[scenario_id]
    agent = RnnAgent(model, args)
    agent.feed_context(ctx.flatten(),
                       belief_constructor,
                       num_markables_to_force=num_markables[0] if force_num_markables else None)
    agent.real_ids = real_ids
    agent.agent_id = agents[0]

    conv = []
    speaker = []
    words_left = 5000

    sentence_ix = 0

    assert len(inpts) == len(tgts)

    while sentence_ix < len(inpts):
        words_og = [corpus.word_dict.idx2word[ix] for ix in inpts[sentence_ix].flatten().numpy()]
        
        if ref_inpts[sentence_ix] is not None:
            dots_mentioned = (ref_tgts[sentence_ix].sum(dim=1) > 0)
            dots_mentioned_per_ref = ref_tgts[sentence_ix]
            this_num_markables = num_markables[sentence_ix]
    #         display_attn(scenario, dots_mentioned.float(), agent.agent_id, name='dots_mentioned')
            mentions = [dots_mentioned_per_ref[:,mention_ix].float()  for mention_ix in range(dots_mentioned_per_ref.size(1))]
            nm_out, _, nm_num_markables = agent.next_mention_outs[-1]
#             print('nm_num_markables: {}'.format(nm_num_markables))
#             print('num_markables: {}'.format(this_num_markables))
            if display_mentions and nm_out is not None:
#                 print(nm_out[0].size())
#                 dummy_targets = torch.zeros_like(nm_out[0].size())
                dummy_targets = torch.zeros((1, nm_num_markables[0].item(), 7)).long()
                _, nm_preds, _ = reference_predictor.forward(
                    True, dummy_targets, nm_out, nm_num_markables
                )
#                 print(nm_preds.size())
                display_attns(scenario, nm_preds.squeeze(1).float(), agent.agent_id, "to_mention_pred")
            else:
                print("empty to_mention_pred")
            display_attns(scenario, mentions, agent.agent_id, "to_mention_true")
#             for mention_ix in range(dots_mentioned_per_ref.size(1)):
#                 display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), agent.agent_id, name=f'dots_mentioned_{mention_ix}')
        else:
            dots_mentioned = torch.zeros(1, 7).bool()
            dots_mentioned_per_ref = torch.zeros(1, 0, 7).bool()
            this_num_markables = torch.LongTensor([0])

        if is_self[sentence_ix]:
            if inference == 'sample':
                for sample_ix in range(candidates):
                    pred_outs, _, _, _, extra = agent.model.write(
                        agent.state, 
                        words_left, sample_temperature,
                        dots_mentioned=dots_mentioned,
                        dots_mentioned_per_ref=dots_mentioned_per_ref,
                        num_markables=this_num_markables
                    )
                    print('sample {}\t{}'.format(sample_ix, ' '.join(agent._decode(pred_outs, agent.model.word_dict))))
            elif inference in ['beam', 'gumbel_beam']:
                _, _, decoded = agent.model.write_beam(
                    agent.state, candidates, words_left, 
                    dots_mentioned=dots_mentioned,
                    dots_mentioned_per_ref=dots_mentioned_per_ref,
                    num_markables=this_num_markables,
                    temperature=sample_temperature if inference=='gumbel_beam' else 1.0,
                    gumbel_noise=inference=='gumbel_beam',
                )
                for cand_ix, utt in enumerate(decoded):
                    print('beam {}\t{}'.format(cand_ix, ' '.join(utt)))
            else:
                raise NotImplementedError(inference)
            pred_outs, _, _, _, extra = agent.model.write(
                agent.state, 
                words_left, argmax_temperature,
                dots_mentioned=dots_mentioned,
                dots_mentioned_per_ref=dots_mentioned_per_ref,
                num_markables=this_num_markables
            )
            print('greedy\t{}'.format(' '.join(agent._decode(pred_outs, agent.model.word_dict))))
    #         print(' '.join(words_og))

            out = agent.write(
                max_words=words_left,
                force_words=[words_og],
                start_token=words_og[0],
                dots_mentioned=dots_mentioned,
                dots_mentioned_per_ref=dots_mentioned_per_ref,
                num_markables=this_num_markables,
                ref_inpt=ref_inpts[sentence_ix],
            )
        
        else:
            out = words_og
            agent.read(words_og,
                        dots_mentioned=dots_mentioned,
                        dots_mentioned_per_ref=dots_mentioned_per_ref,
                        num_markables=this_num_markables,
                        partner_ref_inpt=partner_ref_inpts[sentence_ix],
                        partner_num_markables=partner_num_markables[sentence_ix],
                        next_num_markables_to_force=num_markables[sentence_ix + 1] if (force_num_markables and sentence_ix < len(num_markables) - 1) else None,
                       )
        
#         agent.update_dot_h(ref_inpts[sentence_ix], partner_ref_inpts[sentence_ix],
#                             num_markables[sentence_ix], partner_num_markables[sentence_ix])
            

        print('human\t{}'.format(' '.join(out)))
    
        if is_self[sentence_ix]:
            if ref_inpts[sentence_ix] is not None:
                _, ref_preds, ref_stats = reference_predictor.forward(
                    ref_inpts[sentence_ix], ref_tgts[sentence_ix], agent.ref_outs[-1], this_num_markables
                )
#                 print("ref_preds size:", ref_preds.size())
                display_attns(scenario, ref_preds.squeeze(1).float(), agent.agent_id, "refs_pred")
                display_attns(scenario, ref_tgts[sentence_ix].squeeze(0).float(), agent.agent_id, "refs_true")
#                 print(ref_stats)
            if out[0] != 'YOU':
                utt = ['YOU:'] + out
            else:
                utt = out
        else:
            if partner_ref_inpts[sentence_ix] is not None:
                _, partner_ref_preds, partner_ref_stats = reference_predictor.forward(
                    partner_ref_inpts[sentence_ix], partner_ref_tgts_our_view[sentence_ix],
                    agent.partner_ref_outs[-1], partner_num_markables[sentence_ix]
                )
    #             print(partner_ref_preds)
                display_attns(scenario, partner_ref_preds.squeeze(1).float(), agent.agent_id, "partner_refs_pred")
                display_attns(scenario, partner_ref_tgts_our_view[sentence_ix].squeeze(0).float(), agent.agent_id, "partner_refs_true")
#                 print(partner_ref_stats)
            if out[0] == 'THEM:':
                utt = ['YOU:'] + out[1:]
            else:
                utt = ['YOU:'] + out
    
        print('utt: {}'.format(' '.join(utt[1:])))
        print('detected markables:')
        markables = detect_markables(markable_detector, utt)
        for markable in markables:
            print(markable)
        
        print()
        print('-'*40)
        print()
        words_left -= len(out)
        conv.append(out)

        if is_selection(out):
            # sel_outs should be length 1
            selection_logits, _, _ = agent.sel_outs[-1]
            display_attns(scenario, selection_logits.sigmoid().float(), agent.agent_id, "sel_pred")
            sel_true = torch.zeros(1,7)
            sel_true[0,sel_tgt] = 1
            display_attns(scenario, sel_true, agent.agent_id, "sel_true")
        sentence_ix += 1

In [50]:
pdb off

Automatic pdb calling has been turned OFF


In [67]:
force_contexts_and_mentions(1, inference='beam', candidates=1)

human	THEM: line of 5 dots dark medium at top , large grey below smaller grey under that . small dark under that . medium grey at bottom <eos>
partner_refs_pred


partner_refs_true


utt: line of 5 dots dark medium at top , large grey below smaller grey under that . small dark under that . medium grey at bottom <eos>
detected markables:
({'start': 3, 'end': 17, 'is_self': True, 'text': 'line of 5 dots'}, (1, 4, 5))
({'start': 18, 'end': 29, 'is_self': True, 'text': 'dark medium'}, (5, 6, 7))
({'start': 39, 'end': 49, 'is_self': True, 'text': 'large grey'}, (10, 11, 12))
({'start': 56, 'end': 68, 'is_self': True, 'text': 'smaller grey'}, (13, 14, 15))
({'start': 75, 'end': 79, 'is_self': True, 'text': 'that'}, (16, 16, 17))
({'start': 82, 'end': 92, 'is_self': True, 'text': 'small dark'}, (18, 19, 20))
({'start': 99, 'end': 103, 'is_self': True, 'text': 'that'}, (21, 21, 22))
({'start': 106, 'end': 117, 'is_self': True, 'text': 'medium grey'}, (23, 24, 25))

----------------------------------------

to_mention_pred


to_mention_true


beam 0	i have that too . let 's pick the tiny one . <selection>
greedy	i have that too . let 's pick the tiny one . <selection>
human	i have a small black dot . click that one ? <eos>
refs_pred


refs_true


utt: i have a small black dot . click that one ? <eos>
detected markables:
({'start': 10, 'end': 27, 'is_self': True, 'text': 'a small black dot'}, (3, 6, 7))
({'start': 36, 'end': 44, 'is_self': True, 'text': 'that one'}, (9, 10, 11))

----------------------------------------

human	THEM: ok <selection>
utt: ok <selection>
detected markables:

----------------------------------------

sel_pred


sel_true


In [68]:
force_contexts_and_mentions(1, inference='beam', candidates=1, force_num_markables=False)

human	THEM: line of 5 dots dark medium at top , large grey below smaller grey under that . small dark under that . medium grey at bottom <eos>
partner_refs_pred


partner_refs_true


utt: line of 5 dots dark medium at top , large grey below smaller grey under that . small dark under that . medium grey at bottom <eos>
detected markables:
({'start': 3, 'end': 17, 'is_self': True, 'text': 'line of 5 dots'}, (1, 4, 5))
({'start': 18, 'end': 29, 'is_self': True, 'text': 'dark medium'}, (5, 6, 7))
({'start': 39, 'end': 49, 'is_self': True, 'text': 'large grey'}, (10, 11, 12))
({'start': 56, 'end': 68, 'is_self': True, 'text': 'smaller grey'}, (13, 14, 15))
({'start': 75, 'end': 79, 'is_self': True, 'text': 'that'}, (16, 16, 17))
({'start': 82, 'end': 92, 'is_self': True, 'text': 'small dark'}, (18, 19, 20))
({'start': 99, 'end': 103, 'is_self': True, 'text': 'that'}, (21, 21, 22))
({'start': 106, 'end': 117, 'is_self': True, 'text': 'medium grey'}, (23, 24, 25))

----------------------------------------

to_mention_pred


to_mention_true


beam 0	i have that too . let 's pick the tiny one . <selection>
greedy	i have that too . let 's pick the tiny one . <selection>
human	i have a small black dot . click that one ? <eos>
refs_pred


refs_true


utt: i have a small black dot . click that one ? <eos>
detected markables:
({'start': 10, 'end': 27, 'is_self': True, 'text': 'a small black dot'}, (3, 6, 7))
({'start': 36, 'end': 44, 'is_self': True, 'text': 'that one'}, (9, 10, 11))

----------------------------------------

human	THEM: ok <selection>
utt: ok <selection>
detected markables:

----------------------------------------

sel_pred


sel_true


In [69]:
force_contexts_and_mentions(2, candidates=1, force_num_markables=False)

human	THEM: i have medium size black dot at south <eos>
partner_refs_pred


partner_refs_true


utt: i have medium size black dot at south <eos>
detected markables:
({'start': 10, 'end': 31, 'is_self': True, 'text': 'medium size black dot'}, (3, 6, 7))

----------------------------------------

to_mention_pred


to_mention_true


beam 0	i have a black dot with a small light grey dot to the right of it <eos>
greedy	i have a black dot with a small light grey dot to the right of it <eos>
human	i have a medium sized black dot with a lighter and slightly smaller dot below and to the right of it <eos>
refs_pred


...

to_mention_true


beam 0	i see that . let 's pick the black dot . <selection>
greedy	i see that . let 's pick the black dot . <selection>
human	yes . i have the same . let 's pick the black dot <selection>
refs_pred


refs_true


utt: yes . i have the same . let 's pick the black dot <selection>
detected markables:
({'start': 16, 'end': 24, 'is_self': True, 'text': 'the same'}, (5, 6, 7))
({'start': 39, 'end': 52, 'is_self': True, 'text': 'the black dot'}, (11, 13, 14))

----------------------------------------

sel_pred


sel_true


In [70]:
force_contexts_and_mentions(2, candidates=1)

human	THEM: i have medium size black dot at south <eos>
partner_refs_pred


partner_refs_true


utt: i have medium size black dot at south <eos>
detected markables:
({'start': 10, 'end': 31, 'is_self': True, 'text': 'medium size black dot'}, (3, 6, 7))

----------------------------------------

to_mention_pred


to_mention_true


beam 0	i have a black dot with a small light grey dot to the right of it <eos>
greedy	i have a black dot with a small light grey dot to the right of it <eos>
human	i have a medium sized black dot with a lighter and slightly smaller dot below and to the right of it <eos>
refs_pred


...

to_mention_true


beam 0	i see that . let 's pick the black dot . <selection>
greedy	i see that . let 's pick the black dot . <selection>
human	yes . i have the same . let 's pick the black dot <selection>
refs_pred


refs_true


utt: yes . i have the same . let 's pick the black dot <selection>
detected markables:
({'start': 16, 'end': 24, 'is_self': True, 'text': 'the same'}, (5, 6, 7))
({'start': 39, 'end': 52, 'is_self': True, 'text': 'the black dot'}, (11, 13, 14))

----------------------------------------

sel_pred


sel_true


In [71]:
force_contexts_and_mentions(3, candidates=1)

empty to_mention_pred
to_mention_true


beam 0	i have a large black dot <eos>
greedy	i have a large black dot <eos>
human	i have one large black dot <eos>
refs_pred


refs_true


utt: i have one large black dot <eos>
detected markables:
({'start': 10, 'end': 29, 'is_self': True, 'text': 'one large black dot'}, (3, 6, 7))

----------------------------------------

human	THEM: i have a large dot but i think its dark grey because one next to it is black . i have one tiny light grey dot <eos>
partner_refs_pred


partner_refs_true


...

to_mention_true


beam 0	let 's pick the tiny light grey dot <selection>
greedy	let 's pick the tiny light grey dot <selection>
human	let 's click the small light grey one . <eos>
refs_pred


refs_true


utt: let 's click the small light grey one . <eos>
detected markables:
({'start': 16, 'end': 40, 'is_self': True, 'text': 'the small light grey one'}, (4, 8, 9))

----------------------------------------

human	THEM: ok <selection>
utt: ok <selection>
detected markables:

----------------------------------------

sel_pred


sel_true


In [72]:
force_contexts_and_mentions(4, candidates=1)

human	THEM: do you have 2 dots close in size close together . left one is lighter than the right ? <eos>
partner_refs_pred


partner_refs_true


utt: do you have 2 dots close in size close together . left one is lighter than the right ? <eos>
detected markables:
({'start': 15, 'end': 21, 'is_self': True, 'text': '2 dots'}, (4, 5, 6))
({'start': 53, 'end': 61, 'is_self': True, 'text': 'left one'}, (12, 13, 14))
({'start': 78, 'end': 87, 'is_self': True, 'text': 'the right'}, (17, 18, 19))

----------------------------------------

to_mention_pred


to_mention_true


beam 0	i have a cluster of 4 dots , one of the rest are the same size and the rest are all the same size . the rest are the same size and the bottom one is the darkest <eos>
greedy	i have a cluster of 4 dots , one of the rest are the same size and the rest are all the same size . the rest are the same size and the bottom one is the darkest <eos>
human	do you have mostly dark ones ? there is one med/grey one toward the bottom , small . with a small black one same size to it 's lower right <eos>
refs_pred


...

to_mention_true


beam 0	ok , let 's pick the top one <selection>
greedy	ok , let 's pick the top one <selection>
human	ok lets pick the light upper left one ty <selection>
refs_pred


refs_true


utt: ok lets pick the light upper left one ty <selection>
detected markables:
({'start': 16, 'end': 40, 'is_self': True, 'text': 'the light upper left one'}, (4, 8, 9))

----------------------------------------

sel_pred


sel_true


In [73]:
force_contexts_and_mentions(5)

human	THEM: i have a tiny light gray slightly to the left of 6 o clock with a med size gray dot same color above it <eos>
partner_refs_pred


partner_refs_true


utt: i have a tiny light gray slightly to the left of 6 o clock with a med size gray dot same color above it <eos>
detected markables:
({'start': 10, 'end': 27, 'is_self': True, 'text': 'a tiny light gray'}, (3, 6, 7))
({'start': 67, 'end': 86, 'is_self': True, 'text': 'a med size gray dot'}, (16, 20, 21))
({'start': 104, 'end': 106, 'is_self': True, 'text': 'it'}, (24, 24, 25))

----------------------------------------

to_mention_pred


to_mention_true


beam 0	i do n't have that <eos>
greedy	i do n't have that <eos>
human	i do n't see a match like that . <eos>
refs_pred


...

to_mention_true


beam 0	ok lets click the black one <selection>
greedy	ok lets click the black one <selection>
human	we 'll click to darker one then ? <eos>
refs_pred


refs_true


utt: we 'll click to darker one then ? <eos>
detected markables:
({'start': 19, 'end': 29, 'is_self': True, 'text': 'darker one'}, (5, 6, 7))

----------------------------------------

human	THEM: okay sounds good <selection>
utt: okay sounds good <selection>
detected markables:

----------------------------------------

sel_pred


sel_true


In [74]:
force_contexts_and_mentions(6)

human	THEM: mid size grey dot at 6 o clock <eos>
partner_refs_pred


partner_refs_true


utt: mid size grey dot at 6 o clock <eos>
detected markables:
({'start': 3, 'end': 20, 'is_self': True, 'text': 'mid size grey dot'}, (1, 4, 5))

----------------------------------------

to_mention_pred


to_mention_true


beam 0	i have a large light grey dot at the top of the circle <eos>
greedy	i have a large grey dot at the top <eos>
human	hi , do you have 2 big dots towards the left , upper of the circle ? <eos>
refs_pred


...

to_mention_true


beam 0	yes , let 's pick that one <selection>
greedy	yes , let 's pick that one <selection>
human	the darker one , <unk> from 10 o'clock <selection>
refs_pred


refs_true


utt: the darker one , <unk> from 10 o'clock <selection>
detected markables:
({'start': 3, 'end': 17, 'is_self': True, 'text': 'the darker one'}, (1, 3, 4))

----------------------------------------

sel_pred


sel_true


In [75]:
force_contexts_and_mentions(7)

human	THEM: small dark dot ? <eos>
partner_refs_pred


partner_refs_true


utt: small dark dot ? <eos>
detected markables:
({'start': 3, 'end': 17, 'is_self': True, 'text': 'small dark dot'}, (1, 3, 4))

----------------------------------------

to_mention_pred


to_mention_true


beam 0	i have a cluster of 4 dots that form a triangle <eos>
greedy	i have a cluster of 4 dots that form a triangle <eos>
human	i see 3 dots in a line <eos>
refs_pred


...

partner_refs_true


utt: is the dot on the left small and dark ? <eos>
detected markables:
({'start': 6, 'end': 13, 'is_self': True, 'text': 'the dot'}, (2, 3, 4))

----------------------------------------

beam 0	yes <eos>
greedy	yes <eos>
human	yes <eos>
utt: yes <eos>
detected markables:

----------------------------------------

human	THEM: choose that one <selection>
partner_refs_pred


partner_refs_true


utt: choose that one <selection>
detected markables:
({'start': 10, 'end': 18, 'is_self': True, 'text': 'that one'}, (2, 3, 4))

----------------------------------------

sel_pred


sel_true


In [76]:
force_contexts_and_mentions(8)

empty to_mention_pred
to_mention_true


beam 0	i have a small dark dot at the top of the circle <eos>
greedy	i have a small dark dot at the top of the circle <eos>
human	i have a tiny dark gray on the bottom to the left side <eos>
refs_pred


refs_true


utt: i have a tiny dark gray on the bottom to the left side <eos>
detected markables:
({'start': 10, 'end': 26, 'is_self': True, 'text': 'a tiny dark gray'}, (3, 6, 7))

----------------------------------------

human	THEM: ah yes i see it , it is standing alone for me . <selection>
partner_refs_pred


partner_refs_true


utt: ah yes i see it , it is standing alone for me . <selection>
detected markables:
({'start': 16, 'end': 18, 'is_self': True, 'text': 'it'}, (5, 5, 6))
({'start': 21, 'end': 23, 'is_self': True, 'text': 'it'}, (7, 7, 8))

----------------------------------------

sel_pred


sel_true


In [77]:
force_contexts_and_mentions(9)

human	THEM: do you see one dot darker than all the rest <eos>
partner_refs_pred


partner_refs_true


utt: do you see one dot darker than all the rest <eos>
detected markables:
({'start': 14, 'end': 21, 'is_self': True, 'text': 'one dot'}, (4, 5, 6))
({'start': 34, 'end': 46, 'is_self': True, 'text': 'all the rest'}, (8, 10, 11))

----------------------------------------

to_mention_pred


to_mention_true


beam 0	yes , and it is a little larger and light grey <eos>
greedy	yes , and it is a little larger and light grey <eos>
human	i have a really light circle touching the dotted line bottom left <eos>
refs_pred


...

utt: yes - i think i will select one of those dots - sound good ? <eos>
detected markables:
({'start': 31, 'end': 48, 'is_self': True, 'text': 'one of those dots'}, (8, 11, 12))

----------------------------------------

empty to_mention_pred
to_mention_true


beam 0	yes , i will pick that one <selection>
greedy	yes , i will pick that one <selection>
human	select the bottom one <eos>
refs_pred


refs_true


utt: select the bottom one <eos>
detected markables:
({'start': 10, 'end': 24, 'is_self': True, 'text': 'the bottom one'}, (2, 4, 5))

----------------------------------------

human	THEM: ok <selection>
utt: ok <selection>
detected markables:

----------------------------------------

sel_pred


sel_true


In [78]:
force_contexts_and_mentions(10)

human	THEM: i have four clustered together . bottom right is largest and darkest <eos>
partner_refs_pred


partner_refs_true


utt: i have four clustered together . bottom right is largest and darkest <eos>
detected markables:
({'start': 10, 'end': 14, 'is_self': True, 'text': 'four'}, (3, 3, 4))
({'start': 36, 'end': 48, 'is_self': True, 'text': 'bottom right'}, (7, 8, 9))

----------------------------------------

to_mention_pred


to_mention_true


beam 0	i have a cluster of 3 dots . one is black and the other is light grey and the other is the darkest <eos>
greedy	i have a cluster of 3 dots . one is black and the other is light grey and the other is the darkest <eos>
human	i have three dots in a triangle , the bottom right one is large and dark -- i may have that and the top right one is out of my circle <eos>
refs_pred


...

refs_true


utt: yes that 's the one i had , one of those dots ? <eos>
detected markables:
({'start': 7, 'end': 11, 'is_self': True, 'text': 'that'}, (2, 2, 3))
({'start': 31, 'end': 48, 'is_self': True, 'text': 'one of those dots'}, (9, 12, 13))

----------------------------------------

human	THEM: yes , lets take the big dark one lower right <selection>
partner_refs_pred


partner_refs_true


utt: yes , lets take the big dark one lower right <selection>
detected markables:
({'start': 19, 'end': 35, 'is_self': True, 'text': 'the big dark one'}, (5, 8, 9))

----------------------------------------

sel_pred


sel_true


In [79]:
force_contexts_and_mentions(11)

empty to_mention_pred
to_mention_true


beam 0	i have a large black dot with a larger dark grey dot to the right and slightly above <eos>
greedy	i have a large black dot with a larger dark grey dot to the right and slightly above <eos>
human	i have a large dark dot with a slightly lighter and smaller dot above and to the right <eos>
refs_pred


refs_true


utt: i have a large dark dot with a slightly lighter and smaller dot above and to the right <eos>
detected markables:
({'start': 10, 'end': 26, 'is_self': True, 'text': 'a large dark dot'}, (3, 6, 7))
({'start': 32, 'end': 66, 'is_self': True, 'text': 'a slightly lighter and smaller dot'}, (8, 13, 14))

----------------------------------------

human	THEM: i do too . choose the darker one <selection>
partner_refs_pred


partner_refs_true


utt: i do too . choose the darker one <selection>
detected markables:
({'start': 21, 'end': 35, 'is_self': True, 'text': 'the darker one'}, (6, 8, 9))

----------------------------------------

sel_pred


sel_true


In [58]:
import json
from utils import ContextGenerator

In [43]:
ctx_gen = ContextGenerator('data/onecommon/shared_4.txt')

In [44]:
with open('data/onecommon/shared_4.json', "r") as f:
    scenario_list = json.load(f)
selfplay_scenarios = {scenario['uuid']: scenario for scenario in scenario_list}

In [45]:
scenario = None
for ix, ([scenario_id], (ctx1, ctx2), (ids1, ids2)) in enumerate(ctx_gen.iter()):
#     if scenario_id == 'S_11J8n40u4HRVduwi':
#         print('found it')
#         scenario = scenarios[scenario_id]
#         break
    if ix == 2:
        scenario = selfplay_scenarios[scenario_id]
        print(scenario_id)
        break

S_xM5bXi3lMqXHKKgP


In [46]:
import random

In [ ]:
# selfplay emulation
scenario = selfplay_scenarios[scenario_id]
for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
    agent.feed_context(ctx)
    agent.real_ids = real_ids
    agent.agent_id = agent_id
    
dot_mention_samples = 1
utt_samples = 10
    
writer, reader = [alice, bob]
conv = []
speaker = []
words_left = 5000

num_mentions = 2
num_dots_to_mention = 3

sample_temperature = 0.25
argmax_temperature = 0.005

def make_dots_mentioned_per_ref():
#     # fixed
#     return torch.BoolTensor([[1,1,0,0,0,0,0], [1,0,0,0,0,0,0]]).unsqueeze(0)
#     return torch.BoolTensor([[1,1,0,0,0,0,0]]).unsqueeze(0)

    # random initial and drop
    mentioned = torch.zeros(1,num_mentions,7).bool()
    dots_mentioned = []
    for _ in range(num_dots_to_mention):
        dot_ix = torch.randint(7, (1,))
        mentioned[:,0,dot_ix] = 1
        dots_mentioned.append(dot_ix)
        
    for mention_ix in range(1, num_mentions):
        mentioned[:,mention_ix,random.choice(dots_mentioned)] = 1
    return mentioned
    

    # random 
#     mentioned = torch.zeros(1,num_mentions,7).bool()
#     for mention_ix in range(num_mentions):
#         for _ in range(num_dots_to_mention):
#             dot_ix = torch.randint(7, (1,))
#             mentioned[:,mention_ix,dot_ix] = 1
#     return mentioned

display_context = True

while True:
    dots_mentioned_per_ref = make_dots_mentioned_per_ref()
    dots_mentioned = dots_mentioned_per_ref.max(1).values
    
    num_markables=torch.LongTensor([dots_mentioned_per_ref.size(1)])
    if display_context:
        for mention_ix in range(dots_mentioned_per_ref.size(1)):
            display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
    for us in range(utt_samples):
        outs, _, _, _, extra = writer.model.write(
            writer.ctx,
            writer.ctx_differences,
            writer.ctx_h,
            writer.reader_and_writer_lang_h,
            words_left,
            sample_temperature,
            dots_mentioned=dots_mentioned,
            dots_mentioned_per_ref=dots_mentioned_per_ref,
            num_markables=num_markables,
        )
        print('agent {}; sample {}\t{}'.format(writer.agent_id, us, ' '.join(writer._decode(outs, writer.model.word_dict))))
    print()
        
    num_markables=torch.LongTensor([num_mentions])
    if display_context:
        for mention_ix in range(dots_mentioned_per_ref.size(1)):
            display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
    out = writer.write(
        max_words=words_left,
        dots_mentioned=dots_mentioned,
        dots_mentioned_per_ref=dots_mentioned_per_ref,
        num_markables=num_markables,
        temperature_override=argmax_temperature,
    )
    print('agent {}; argmax\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
    
#     if display_context and writer.extras[-1].get('feed_ctx_attn_prob', None) is not None:
#         display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')
    
#     if display_context and writer.extras[-1].get('word_ctx_attn_prob_mean', None) is not None:
#         display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')
        
#     for ix, ix_attn in enumerate(writer.extras[-1]['word_ctx_attn_probs']):
#         display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))
        
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print('-'*60)
        
    words_left -= len(out)
    conv.append(out)
    speaker.append(writer.agent_id)
    reader.read(out)
    if is_selection(out) or words_left <= 1:
        break
    writer, reader = reader, writer

NameError: name 'selfplay_scenarios' is not defined

> <ipython-input-41-e274016a0281>(2)<module>()
      1 # selfplay emulation
----> 2 scenario = selfplay_scenarios[scenario_id]
      3 for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
      4     agent.feed_context(ctx)
      5     agent.real_ids = real_ids

--KeyboardInterrupt--


In [ ]:
markable_detector = utils.load_model('')

In [92]:
model.args.mark_dots_mentioned

True

In [94]:
validset[0]._asdict().keys()

odict_keys(['ctx', 'inpts', 'tgts', 'ref_inpt', 'ref_tgt', 'sel_tgt', 'scenario_ids', 'real_ids', 'partner_real_ids', 'agents', 'chat_ids', 'sel_idxs', 'lens', 'rev_idxs', 'hid_idxs', 'num_markables', 'is_self', 'partner_ref_inpt', 'partner_ref_tgt_our_view', 'partner_num_markables', 'referent_disagreements', 'partner_referent_disagreements'])

In [100]:
class BlankBeliefConstructor(object):
    def make_beliefs():
        return None

In [102]:
ctx.size()

torch.Size([1, 28])

In [75]:
def selfplay_contexts_and_mentions(scenario, candidates=1,
                                    sample_temperature=0.25, argmax_temperature=0.005,
                                    inference='beam'):
#     scenario = selfplay_scenarios[scenario_id]
    alice = RnnAgent(model, args)
    bob = RnnAgent(model, args)
    
    for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
        agent.feed_context(ctx)
        agent.real_ids = real_ids
        agent.agent_id = agent_id
#     ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
#     [scenario_id], real_ids, partner_real_ids, agents, chat_ids, sel_idxs, \
#     lens, rev_idxs, hid_idxs, num_markables, \
#     is_self, partner_ref_inpts, partner_ref_tgts_our_view, partner_num_markables,\
#     ref_disagreements, partner_ref_disagreements = datasplit[index]
    
    bsz = 1
    num_dots = 7
    
    # assume we've been training for a while, so that the beliefs are out of the warmup period (e.g. for models that switch from true->predicted)
    epoch = 1000
    
    assert [] == model.args.ref_beliefs == model.args.partner_ref_beliefs == model.args.generation_beliefs == model.args.selection_beliefs == model.args.next_mention_beliefs
    
    belief_constructor = BlankBeliefConstructor()
    
    writer, reader = alice, bob
    
    conv = []
    speaker = []
    words_left = 5000

    sentence_ix = 0

    while words_left > 0:
        nm_out, _, this_num_markables = writer.next_mention_outs[-1]
        if display_mentions and nm_out is not None:
            dummy_targets = torch.zeros((1, nm_num_markables[0].item(), 7)).long()
            _, dots_to_mention_per_ref, _ = reference_predictor.forward(
                True, dummy_targets, nm_out, nm_num_markables
            )
            display_attns(scenario, dots_to_mention_per_ref.squeeze(1).float(), writer.agent_id, "to_mention_pred")
            dots_to_mention_per_ref = torch.zeros((1, 0, 7)).long()
        else:
            print("empty to_mention_pred")
        dots_to_mention = dots_to_mention_per_ref.max(1)

        if inference == 'sample':
            for sample_ix in range(candidates):
                pred_outs, _, _, _, extra = writer.model.write(
                    writer.state, 
                    words_left, sample_temperature,
                    dots_mentioned=dots_mentioned,
                    dots_mentioned_per_ref=dots_mentioned_per_ref,
                    num_markables=this_num_markables
                )
                print('sample {}\t{}'.format(sample_ix, ' '.join(writer._decode(pred_outs, writer.model.word_dict))))
        elif inference in ['beam', 'gumbel_beam']:
            _, _, decoded = writer.model.write_beam(
                writer.state, candidates, words_left, 
                dots_mentioned=dots_mentioned,
                dots_mentioned_per_ref=dots_mentioned_per_ref,
                num_markables=this_num_markables,
                temperature=sample_temperature if inference=='gumbel_beam' else 1.0,
                gumbel_noise=inference=='gumbel_beam',
            )
            for cand_ix, utt in enumerate(decoded):
                print('beam {}\t{}'.format(cand_ix, ' '.join(utt)))
        else:
            raise NotImplementedError(inference)
            
        pred_outs, _, _, _, extra = writer.model.write(
            writer.state, 
            words_left, argmax_temperature,
            dots_mentioned=dots_mentioned,
            dots_mentioned_per_ref=dots_mentioned_per_ref,
            num_markables=this_num_markables
        )
            print('greedy\t{}'.format(' '.join(writer._decode(pred_outs, writer.model.word_dict))))
    #         print(' '.join(words_og))

        out = writer.write(
            max_words=words_left,
            dots_mentioned=dots_mentioned,
            dots_mentioned_per_ref=dots_mentioned_per_ref,
            num_markables=this_num_markables,
            ref_inpt=ref_inpts[sentence_ix],
        )
        
        reader.read(words_og,
                    dots_mentioned=dots_mentioned,
                    dots_mentioned_per_ref=dots_mentioned_per_ref,
                    num_markables=this_num_markables,
                    partner_ref_inpt=partner_ref_inpts[sentence_ix],
                    partner_num_markables=partner_num_markables[sentence_ix],
                    next_num_markables_to_force=None,
                   )
        
#         writer.update_dot_h(ref_inpts[sentence_ix], partner_ref_inpts[sentence_ix],
#                             num_markables[sentence_ix], partner_num_markables[sentence_ix])

        if is_self[sentence_ix]:
            if ref_inpts[sentence_ix] is not None:
                _, ref_preds, ref_stats = reference_predictor.forward(
                    ref_inpts[sentence_ix], ref_tgts[sentence_ix], writer.ref_outs[-1], this_num_markables
                )
#                 print("ref_preds size:", ref_preds.size())
                display_attns(scenario, ref_preds.squeeze(1).float(), writer.agent_id, "refs_pred")
                display_attns(scenario, ref_tgts[sentence_ix].squeeze(0).float(), writer.agent_id, "refs_true")
#                 print(ref_stats)
        else:
            if partner_ref_inpts[sentence_ix] is not None:
                _, partner_ref_preds, partner_ref_stats = reference_predictor.forward(
                    partner_ref_inpts[sentence_ix], partner_ref_tgts_our_view[sentence_ix],
                    writer.partner_ref_outs[-1], partner_num_markables[sentence_ix]
                )
    #             print(partner_ref_preds)
                display_attns(scenario, partner_ref_preds.squeeze(1).float(), writer.agent_id, "partner_refs_pred")
                display_attns(scenario, partner_ref_tgts_our_view[sentence_ix].squeeze(0).float(), writer.agent_id, "partner_refs_true")
#                 print(partner_ref_stats)

        print()
        print('-'*40)
        print()
        words_left -= len(out)
        conv.append(out)
        if is_selection(out):
            # sel_outs should be length 1
            selection_logits, _, _ = writer.sel_outs[-1]
            display_attns(scenario, selection_logits.sigmoid().float(), writer.agent_id, "sel_pred")
            sel_true = torch.zeros(1,7)
            sel_true[0,sel_tgt] = 1
            display_attns(scenario, sel_true, writer.agent_id, "sel_true")
        sentence_ix += 1